In [137]:
%load_ext autoreload 
%autoreload 2
%reload_ext autoreload

from p1_graphs import *
import altair as alt
data = get_accident_data(fname="dataset_v1.csv",sample=False)
accident_data = get_weather_data(data,fname = "weather.csv")

data.head()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


c:\Users\PcCom\Documents\GitHub\VI_P2\p1_graphs.py:115: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ny_df["x"] = hex_buroughs.centroid.x
c:\Users\PcCom\Documents\GitHub\VI_P2\p1_graphs.py:116: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ny_df["y"] = hex_buroughs.centroid.y
c:\Users\PcCom\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3460: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


,index_left,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,VEHICLE TYPE CODE 1,date,weekday,covid,BoroName,index,BoroCode,Shape_Leng,Shape_Area
0,47,2018-08-31T00:00:00Z,7:55,BRONX,40.827168,-73.870125,Sedan,2018-08-31,weekday,before,Bronx,4,2,464392.991824,1.186925e+09
0,36372,2018-08-14T00:00:00Z,9:20,NaN,40.823387,-73.880340,Suv,2018-08-14,weekday,before,Bronx,4,2,464392.991824,1.186925e+09
0,36376,2018-08-28T00:00:00Z,9:10,BRONX,40.805016,-73.921090,Suv,2018-08-28,weekday,before,Bronx,4,2,464392.991824,1.186925e+09
0,36383,2018-08-03T00:00:00Z,13:00,BRONX,40.834423,-73.879265,Sedan,2018-08-03,weekday,before,Bronx,4,2,464392.991824,1.186925e+09
0,36386,2018-08-07T00:00:00Z,2:25,BRONX,40.902935,-73.849340,Suv,2018-08-07,weekday,before,Bronx,4,2,464392.991824,1.186925e+09


In [89]:
accident_data.columns

Index(['index_left', 'CRASH DATE', 'CRASH TIME', 'BOROUGH', 'LATITUDE',
       'LONGITUDE', 'VEHICLE TYPE CODE 1', 'date', 'weekday', 'covid',
       'BoroName', 'index', 'BoroCode', 'Shape_Leng', 'Shape_Area', 'datetime',
       'conditions'],
      dtype='object')

In [86]:
accident_data.head()

,index_left,CRASH DATE,CRASH TIME,BOROUGH,LATITUDE,LONGITUDE,VEHICLE TYPE CODE 1,date,weekday,covid,BoroName,index,BoroCode,Shape_Leng,Shape_Area,datetime,conditions
0,47,2018-08-31T00:00:00Z,7:55,BRONX,40.827168,-73.870125,Sedan,2018-08-31,weekday,before,Bronx,4,2,464392.991824,1.186925e+09,2018-08-31,"Rain, Overcast"
1,42536,2018-08-31T00:00:00Z,22:00,BRONX,40.837997,-73.921036,Sedan,2018-08-31,weekday,before,Bronx,4,2,464392.991824,1.186925e+09,2018-08-31,"Rain, Overcast"
2,42575,2018-08-31T00:00:00Z,16:09,BRONX,40.847515,-73.900230,Sedan,2018-08-31,weekday,before,Bronx,4,2,464392.991824,1.186925e+09,2018-08-31,"Rain, Overcast"
3,42935,2018-08-31T00:00:00Z,21:05,BRONX,40.819523,-73.901950,Sedan,2018-08-31,weekday,before,Bronx,4,2,464392.991824,1.186925e+09,2018-08-31,"Rain, Overcast"
4,42979,2018-08-31T00:00:00Z,13:25,BRONX,40.854183,-73.891050,Sedan,2018-08-31,weekday,before,Bronx,4,2,464392.991824,1.186925e+09,2018-08-31,"Rain, Overcast"


In [138]:
mapa = get_map()
ny_df, bur = get_buroughs(mapa)
# normalize bur and accident data BoroName
bur = bur.reset_index()
accident_data.head()
# create properties.name column equal to BoroName
accident_data['properties.name'] = accident_data['BoroName']

#filter types of vehicles 
accident_data['VEHICLE TYPE CODE 1'] = accident_data['VEHICLE TYPE CODE 1'].replace(['Fire Truck'],'FIRE')
accident_data['VEHICLE TYPE CODE 1'] = accident_data['VEHICLE TYPE CODE 1'].replace(['Ambulance'],'AMBULANCE')
accident_data['VEHICLE TYPE CODE 1'] = accident_data['VEHICLE TYPE CODE 1'].replace(['Taxi'],'TAXI')
accident_data = accident_data[accident_data['VEHICLE TYPE CODE 1'].isin(['FIRE','AMBULANCE','TAXI'])]

# create HOUR column from CRASH TIME
accident_data["CRASH TIME"] = pd.to_datetime(accident_data["CRASH TIME"])
accident_data['HOUR'] = accident_data['CRASH TIME'].dt.hour



accident_data['weekday'] = accident_data['date'].dt.weekday
# make weekend column
accident_data['weekend'] = accident_data['weekday'].apply(lambda x: 1 if x > 4 else 0)

# make column with week number
accident_data['week'] = accident_data['date'].dt.week
# month column
accident_data['month'] = accident_data['date'].dt.month

# for each month get the minimum week number
min_week = accident_data.groupby(['month'])['week'].min().reset_index()
# merge with accident data
accident_data = pd.merge(accident_data,min_week,on='month',how='left')
accident_data['week'] = accident_data['week_x'] - accident_data['week_y'] + 1


c:\Users\PcCom\Documents\GitHub\VI_P2\p1_graphs.py:115: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ny_df["x"] = hex_buroughs.centroid.x
c:\Users\PcCom\Documents\GitHub\VI_P2\p1_graphs.py:116: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  ny_df["y"] = hex_buroughs.centroid.y
C:\Users\PcCom\AppData\Local\Temp\ipykernel_14536\1304337895.py:26: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  accident_data['week'] = accident_data['date'].dt.week


In [139]:
colors = {"bg": "#eff0f3", "col1": "#d8b365", "col2": "#5ab4ac"}
w = 600
h = 400
ratio = 0.2
# accident_data = get_weather_data(data,fname = "weather.csv")
print(accident_data.columns)
selection_cond = alt.selection_multi(on="click", empty="all", fields=["conditions"])
selection_buro = alt.selection_point(fields=["properties.name"], empty="all")
month_dropdown = alt.binding_select(options=[[6,7,8],6,7,8])
selection_month = alt.selection_point(fields=['month'], bind=month_dropdown)
selection_vehicle = alt.selection_multi(on="click", empty="all", fields=["VEHICLE TYPE CODE 1"])
time_brush = alt.selection_interval(encodings=['x'])


weekday_dropdown = alt.binding_select(options=[[0,1,2,3,4],[5,6]],name='weekday',labels=['weekday','weekend'])
selection_weekday = alt.selection_point(fields=['weekday'], bind=weekday_dropdown)

selection_day = alt.selection_point(fields=['weekday','week','month'])
# choropleth, _ = plot_map(hex_data, mapa, ny_df, bur,selection_buro,selection_cond)


Index(['index_left', 'CRASH DATE', 'CRASH TIME', 'BOROUGH', 'LATITUDE',
       'LONGITUDE', 'VEHICLE TYPE CODE 1', 'date', 'weekday', 'covid',
       'BoroName', 'index', 'BoroCode', 'Shape_Leng', 'Shape_Area', 'datetime',
       'conditions', 'properties.name', 'HOUR', 'weekend', 'week_x', 'month',
       'week_y', 'week'],
      dtype='object')


c:\Users\PcCom\anaconda3\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_multi' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
c:\Users\PcCom\anaconda3\lib\site-packages\altair\vegalite\v5\api.py:394: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(


In [142]:
w=1000
geo_view=plot_map(accident_data, selection_cond, selection_buro, selection_month, selection_weekday, selection_vehicle, time_brush,   w=w,ratio=.9)
weather = weather_chart(accident_data,selection_buro,selection_cond,selection_month,selection_weekday, selection_vehicle, time_brush,  w=w*0.8,ratio=0.8)
calendar = calendar_chart(accident_data,selection_buro,selection_cond,selection_month,selection_weekday, selection_vehicle, time_brush,  w=w*0.2)
vehicles = vehicle_chart(accident_data, selection_buro,selection_cond, selection_month, selection_weekday, selection_vehicle, time_brush)
time_of_day = time_of_day_chart(accident_data, selection_buro,selection_cond, selection_month, selection_weekday, selection_vehicle, time_brush)


# .configure_scale(
#     bandPaddingInner=.1).add_params(date_selector,month_selection, weekday_selection)

geo_view & (weather | calendar)  & (vehicles | time_of_day)

alt.VConcatChart(...)

In [94]:
accident_data["HOUR"]

0       19
1       22
2       19
3        9
4       15
        ..
2828    17
2829    19
2830    16
2831    16
2832    14
Name: HOUR, Length: 2833, dtype: int64

####################################

In [136]:
time_brush = alt.selection_interval(encodings=['x'], empty='all')


base = alt.Chart(accident_data, width=600).mark_area(size=3, opacity = 0.3, interpolate="linear", color = "steelblue").encode(
    x=alt.X("HOUR:T"),
    y=alt.Y("count()").stack(None, title="Accidents per hour")
    )

background = base.add_selection(time_brush)
selected = base.transform_filter(time_brush).mark_area(color = "steelblue")

background + selected


c:\Users\PcCom\anaconda3\lib\site-packages\altair\vegalite\v5\api.py:394: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(
c:\Users\PcCom\anaconda3\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.LayerChart(...)